# DeepSphere using SHREC17 dataset
## Benchmark with Cohen method S2CNN[[1]](http://arxiv.org/abs/1801.10130) and Esteves method[[2]](http://arxiv.org/abs/1711.06721)

Multi-class classification of 3D objects, using the interesting property of rotation equivariance.

The 3D objects are projected on a unit sphere.

Several features are collected:
* projection ray length (from sphere border to intersection [0, 2])
* cos/sin with surface normal
* same features using the convex hull of the 3D object

### Equiangular sampling scheme
Use of the equiangular sampling scheme used by Cohen to prove the flexibility of DeepSphere

## 0.1 Load libs

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import shutil
import sys
sys.path.append('../..')

os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # change to chosen GPU to use, nothing if work on CPU

import numpy as np
import time
import matplotlib.pyplot as plt
import healpy as hp

In [ ]:
from deepsphere import models, experiment_helper, plot, utils
from deepsphere.data import LabeledDatasetWithNoise, LabeledDataset
import hyperparameters

from load_shrec import fix_dataset, Shrec17Dataset, Shrec17DatasetCache, Shrec17DatasetTF

## 0.2 Define parameters

In [ ]:
bw = 64
experiment = 'equiangular'
experiment_type = 'CNN' # 'FCN'
ename = '_'+experiment_type
datapath = '../../../data/shrec17/' # localisation of the .obj files

In [ ]:
noise_dataset = True    # use perturbed dataset (Cohen and Esteves do the same)
augmentation = 1        # number of element per file (1 = no augmentation of dataset)

## 1 Load dataset

In [ ]:
# if datasets are already downloaded but not preprocessed
fix = False
if fix:
    fix_dataset(datapath+'val_perturbed')
    fix_dataset(datapath+'test_perturbed')

download dataset if True, preprocess data and store it in npy files, and load it in a dataset object

In [ ]:
download = False
train_dataset = Shrec17DatasetTF(datapath, 'train', perturbed=noise_dataset, download=download, nside=bw, 
                                    augmentation=augmentation, nfile=None, experiment = experiment)


In [ ]:
val_dataset = Shrec17DatasetCache(datapath, 'val', perturbed=noise_dataset, download=download, nside=bw, 
                                 augmentation=1, nfile=None, experiment = experiment)

In [ ]:
# from tqdm import tqdm
# data_iter = train_dataset.iter(32)
# steps = int(train_dataset.N / 32)
# for i in tqdm(range(steps)):
#     next(data_iter)

In [ ]:
# data_iter = val_dataset.iter(32)
# steps = int(val_dataset.N / 32)
# for i in tqdm(range(steps)):
#     next(data_iter)

## 1.1 Preprocess the dataset

Shuffle the training dataset and print the classes distribution

In [ ]:
nclass = train_dataset.nclass
num_elem = train_dataset.N
print('number of class:',nclass,'\nnumber of elements:',num_elem)

#### Plot sphere images

Show what the projection looks like for the first two features

In [ ]:
dataset = train_dataset.get_tf_dataset(1)


In [ ]:
import tensorflow as tf
from tqdm import tqdm

#dataset = tf_dataset_file(datapath, dataset, file_pattern, 32, Nside, augmentation)
data_next = dataset.make_one_shot_iterator().get_next()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
steps = train_dataset.N 
cm = plt.cm.RdBu_r
cm.set_under('w')
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
#     try:
    for i in range(steps):
#         print(i)
        out = sess.run(data_next)
#         print(out[0].shape)
        img, label = out
        im1 = img[0,:,0]
        cmin = np.min(img[:,:,0])
        cmax = np.max(img[:,:,0])
        plt.imshow(im1.reshape((2*bw,2*bw)), cmap=cm, vmin = cmin, vmax = cmax)
#         print(label)
        break

## 2 Classification using DeepSphere

Use of the Dataset object used for other DeepSphere experiments

In [ ]:
EXP_NAME = 'shrec17_equiangular_{}aug_{}bw{}'.format(augmentation, bw, ename)
#EXP_NAME = "shrec17_40sim_32sides_0noise_FCN"
#EXP_NAME = 'plop'

Load model with hyperparameters chosen.
For each experiment, a new EXP_NAME is chosen, and new hyperparameters are store.
All informations are present 'DeepSphere/Shrec17/experiments.md'
The fastest way to reproduce an experiment is to revert to the commit of the experiment to load the correct files and notebook

Adding a layer in the fully connected can be beneficial

In [ ]:
params = hyperparameters.get_params_shrec17_equiangular(num_elem, EXP_NAME, nclass, architecture=experiment_type)
params["tf_dataset"] = train_dataset.get_tf_dataset(params["batch_size"])
model = models.deepsphere(**params)

In [ ]:
shutil.rmtree('summaries/{}/'.format(EXP_NAME), ignore_errors=True)
shutil.rmtree('checkpoints/{}/'.format(EXP_NAME), ignore_errors=True)

Find a correct learning rate

In [ ]:
# backup = params.copy()

# params, learning_rate = utils.test_learning_rates(params, training.N, 1e-6, 1e-1, num_epochs=20)

# shutil.rmtree('summaries/{}/'.format(params['dir_name']), ignore_errors=True)
# shutil.rmtree('checkpoints/{}/'.format(params['dir_name']), ignore_errors=True)

# model = models.deepsphere(**params)
# _, loss_validation, _, _ = model.fit(training, validation)

# params.update(backup)

# plt.semilogx(learning_rate, loss_validation, '.-')

0.9 seems to be a good learning rate for SGD with current parameters

## 2.2 Train Network

In [ ]:
print("the number of parameters in the model is: {:,}".format(model.get_nbr_var()))

In [ ]:
accuracy_validation, loss_validation, loss_training, t_step, t_batch = model.fit(train_dataset, val_dataset, use_tf_dataset=True, cache=True)

In [ ]:
plot.plot_loss(loss_training, loss_validation, t_step, params['eval_frequency'])

Remarks

In [ ]:
# model.evaluate(train_dataset, None, cache='TF')

In [ ]:
model.evaluate(val_dataset, None, cache=True)

In [ ]:
probabilities,_ = model.probs(val_dataset, nclass, cache=True)
# if augmentation>1:
#     probabilities = probabilities.reshape((-1,augmentation,nclass))
#     probabilities = probabilities.mean(axis=1)
#     ids_val = ids_val[::repeat]
predictions = np.argmax(probabilities, axis=1)

In [ ]:
ids_val = val_dataset.get_ids()

In [ ]:
# for every file, find every object with the same class, sorted by most relevance
os.makedirs(os.path.join(datapath,'results_equiangular/val_perturbed'), exist_ok=True)
for i,_id in enumerate(ids_val):
    idfile = os.path.join(datapath,'results_equiangular/val_perturbed',_id)
    # predictions batchxclass
    # pred_class batch == predictions
    retrieved = [(probabilities[j, predictions[j]], ids_val[j]) for j in range(len(ids_val)) if predictions[j] == predictions[i]]
    retrieved = sorted(retrieved, reverse=True)
    retrieved = [i for _, i in retrieved]
    with open(idfile, "w") as f:
        f.write("\n".join(retrieved))

NaN appears if remove i==j case

## test network

In [ ]:
test_dataset = Shrec17DatasetCache(datapath, 'test', perturbed=noise_dataset, download=download, nside=bw, 
                              augmentation=augmentation, nfile=None, experiment=experiment)

In [ ]:
model.evaluate(test_dataset, None, cache=True)

In [ ]:
ids_test = test_dataset.get_ids()

In [ ]:
labels_test = test_dataset.get_labels()

In [ ]:
probabilities,_ = model.probs(test_dataset, nclass, cache=True)
if augmentation>1:
    probabilities = probabilities.reshape((-1,augmentation,nclass))
    probabilities = probabilities.mean(axis=1)
    ids_test = ids_test[::augmentation]
predictions = np.argmax(probabilities, axis=1)

write to file

In [ ]:
# for every file, find every object with the same class, sorted by most relevance
os.makedirs(os.path.join(datapath,'results_equiangular/test_perturbed'), exist_ok=True)
for i, _id in enumerate(ids_test):
    idfile = os.path.join(datapath,'results_equiangular/test_perturbed',_id)
    # predictions batchxclass
    # pred_class batch == predictions
    retrieved = [(probabilities[j, predictions[j]], ids_test[j]) for j in range(len(ids_test)) if predictions[j] == predictions[i]]
    retrieved = sorted(retrieved, reverse=True)
    retrieved = [i for _, i in retrieved]
    with open(idfile, "w") as f:
        f.write("\n".join(retrieved))

Why not working?

In [ ]:
def _print_histogram(nclass, labels_train, labels_val=None):
    if labels_train is None:
        return
    import matplotlib.pyplot as plt
    from collections import Counter
    hist_train=Counter(labels_train)
#         for i in range(self.nclass):
#             hist_train.append(np.sum(labels_train == i))
    labels, values = zip(*hist_train.items())
    indexes = np.asarray(labels)
#     miss = set(indexes) - set(labels)
#     if len(miss) is not 0:
#         hist_train.update({elem:0 for elem in miss})
#     labels, values = zip(*hist_train.items())
    width = 1
    plt.bar(labels, values, width)
    plt.title("labels distribution")
    #plt.xticks(indexes + width * 0.5, labels)
    if labels_val is not None:
        hist_val=Counter(labels_val)
        plt.figure()
        labels, values = zip(*hist_val.items())
        indexes = np.asarray(labels)
        width = 1
        plt.bar(indexes, values, width)
        plt.title("validation labels distribution")
    plt.show()

In [ ]:
_print_histogram(55, labels_test)
_print_histogram(55, predictions)

## Shrec projection outside

In [ ]:
from SHREC17.load_shrec import plot_healpix_projection, cache_healpix_projection

In [ ]:
im1=plot_healpix_projection('../data/shrec17/train_perturbed/000018.obj',128, outside=False, rotp=False, multiple=False, rot=(30,333,275))
plt.savefig("./figures/lamp_sphere_side_000018.png", bbox_inches='tight', transparent=True)

In [ ]:
from matplotlib import transforms
from scipy.ndimage import rotate
fig = plt.figure()
ax = plt.subplot(111)
im = plt.imread('./figures/000018.png')
im = im[500:1600,1000:2800,:]
im = rotate(im, -33)
# print(im.shape)
im = im[:1900,100:2000,:]
circle = plt.Circle((im.shape[0]//2, im.shape[1]//2), 750, color='black', fill=False)
ax.add_artist(circle)
ax.plot([im.shape[0]//2, im.shape[1]//2], [im.shape[0]//2-750, im.shape[1]//2+750], 'k-', color='r', zorder=0)
ax.plot([im.shape[0]//2-750, im.shape[1]//2-300], [im.shape[0]//2, im.shape[1]//2], 'k-', color='r', zorder=0)
ax.plot([im.shape[0]//2+400, im.shape[1]//2+750], [im.shape[0]//2, im.shape[1]//2], 'k-', color='r', zorder=0)
ax.plot([im.shape[0]//2-380/np.sqrt(2), im.shape[1]//2+750/np.sqrt(2)], [im.shape[0]//2-380/np.sqrt(2), im.shape[1]//2+750/np.sqrt(2)], 'k-', color='r', zorder=0)
ax.plot([im.shape[0]//2-750/np.sqrt(2), im.shape[1]//2-450/np.sqrt(2)], [im.shape[0]//2-750/np.sqrt(2), im.shape[1]//2-450/np.sqrt(2)], 'k-', color='r', zorder=0)
ax.plot([im.shape[0]//2-200/np.sqrt(2), im.shape[1]//2-750/np.sqrt(2)], [im.shape[0]//2+200/np.sqrt(2), im.shape[1]//2+750/np.sqrt(2)], 'k-', color='r', zorder=0)
ax.plot([im.shape[0]//2+750/np.sqrt(2), im.shape[1]//2+150/np.sqrt(2)], [im.shape[0]//2-750/np.sqrt(2), im.shape[1]//2-150/np.sqrt(2)], 'k-', color='r', zorder=0)
ax.imshow(im, zorder=100)
ax.axis('off')
# ax.get_yaxis().set_visible(False)
# ax.get_xaxis().set_visible(False)
plt.savefig("./figures/lamp_000018.svg", bbox_inches='tight', transparent=True)

Creates map of shapes projected on the outside of the sphere
* these maps are random part of the sphere graph, and cannot be used directly, as all graphs will be different

In [ ]:
## generates new maps
# cache_healpix_projection('../data/shrec17/', 'test', 32, repeat=3, outside='equator', rot=False)

In [ ]:
train_dataset = Shrec17DatasetCache(datapath, 'train', perturbed=noise_dataset, download=False, nside=32, 
                                  experiment='equator', augmentation=1, nfile=None)

In [ ]:
data_iter = train_dataset.iter(1)
data, label = next(data_iter)

In [ ]:
im1 = data[0]

In [ ]:
im1[np.where(im1==0.)]=np.nan

In [ ]:
cm = plt.cm.RdBu_r
cm.set_under('w')
cmin = np.nanmin(im1)
cmax = np.nanmax(im1)
hp.orthview(im1, title='000003', nest=True, cmap=cm, min=cmin, max=cmax)

In [ ]:
indexes = np.where(np.invert(np.isnan(im1)))[0]
npix = len(indexes)

In [ ]:
from deepsphere import utils

In [ ]:
g = utils.healpix_graph(nside=32, indexes=indexes)

In [ ]:
g.plot(vertex_size=10)

In [ ]:
g.plot_signal(im1[np.where(~np.isnan(im1))], edges=False, vertex_size=10)

In [ ]:
plt.plot(g.e[:16], 'o')

In [ ]:
g.compute_laplacian("normalized")
#g.compute_fourier_basis(recompute=True)
g.set_coordinates(g.U[:,1:4])
g.plot(vertex_size=10)